In [1]:
import xarray as xr
import csv
import pandas as pd
import numpy as np
import os
import pickle as pkl
from datetime import datetime, date, timedelta
import sys
import matplotlib.pyplot as plt
import netCDF4 as nc
import glob
from matplotlib.pyplot import figure
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as mcolors
from numpy import trapz
import pickle
import gsw
from holteandtalley import HolteAndTalley
from FUNCTIONS_HYCOM import set_regions, set_regions_tags

In [2]:
# Define parameters
# define start and end years
start_year = 2011 # 2009
end_year = 2015 # 2018

years = ['2011', '2012', '2013', '2014', '2015']
var_names_download = ['temp', 'salt']
var_names = ['temperature', 'salinity']

# HYCOM_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/5_HYCOM_near_TC_53x_pkl_NEW_BOX_' + folder_tag + '/'

# forcing_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/FORCING_alltimes_singleevents/'
# HYCOM_save_dir  = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/6_HYCOM_near_TC_53x_pkl_NEW_BOX_postproc_' + folder_tag + '/'

tag_file = 'HYCOM_53X'

wind_threshold = 32.9244
precip_treshold = 0.0025

max_depth_index_sel = 22

days_from_event_center = 2
days_before_sel_date = 7
days_after_sel_date = 28

flag_plots = False


In [3]:
# List all the regions

# lat_max, lat_min, lon_max, lon_min for every region
regions = set_regions()
regions_tags = set_regions_tags(regions)
print(regions_tags)


['WPacific_1of3', 'EPacific', 'NorthAtlantic', 'Indian', 'SWPacific_1of2', 'SWPacific_2of2', 'WPacific_2of3', 'WPacific_3of3']


In [4]:
def open_forcing(forcing_name, forcing_dir):
    df_forcing = pd.read_csv(forcing_dir + forcing_name)
    print(forcing_name)

    # Convert lon from [0,360] to [-180,180]
    df_forcing.lon = (df_forcing.lon + 180) % 360 - 180

    # Rename columns to match old version
    df_forcing.rename(columns={'lat': 'LAT', 'lon': 'LONG', 'wind_speed': 'WIND', 'time': 'DATETIME', 'precipitation': 'PRECIPITATION'}, inplace=True)

    # create mask based on start_year
    dfs_mask_1 = pd.to_datetime(df_forcing.DATETIME).dt.year>=start_year
    # apply mask start_year to df
    dfs_1 = df_forcing[dfs_mask_1].reset_index()

    # create mask based on end_year
    dfs_mask_2 = pd.to_datetime(dfs_1.DATETIME).dt.year<=end_year
    # apply mask end_year to df
    dfs_select = dfs_1[dfs_mask_2].reset_index()
    dfs_select.shape
    dfs_select['DATETIME'] = pd.to_datetime(dfs_select['DATETIME'])
    return(dfs_select)


def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


### Open forcing file with selected hurricane events, with WS and precipitation data

### For each selected hurricane event:
### 1) load forcing time series
### 2) load HYCOM time series
### 3) remove seasonal cycle from HYCOM data and calculate anomalies
### 4) keep only data within +- 7 days of event being considered
### 5) plot HYCOM anomalies of temperature and salinity as a function of depth and time, and add forcing WS and precipitation too
### 6) plot HYCOM absolute values of temperature and salinity as a function of depth and time

In [5]:
# ds_anom and variables_anom ARE COMMENTED FOR NOW - might need to change in the future
# not commented anymore as of April 4 2023

In [ ]:
# cell without precipitation and wind - CLEAN CELL THAT WORKS AS OF APRIL 13
for r,r_tag in zip(regions, regions_tags):
    lat_min = r[1]
    lat_max = r[0]
    lon_min = r[3]
    lon_max = r[2]
    HYCOM_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/5_HYCOM_near_TC_53x_pkl_NEW_BOX_' + r_tag + '/'
    forcing_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/FORCING_alltimes_singleevents/'
    HYCOM_save_dir  = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/6_HYCOM_near_TC_53x_pkl_NEW_BOX_postproc_' + r_tag + '/'

    # Open .csv file
    df_forcing = pd.read_csv('df_forcing_hur_events_NEW_BOX_with_functions_' + str(lat_min) + '_' + str(lat_max) + '_' + str(lon_min) + '_' + str(lon_max) + '.csv')
    
    
    # Convert lon from [0,360] to [-180,180]
    #df_forcing.LONG = (df_forcing.LONG + 180) % 360 - 180
    for i in np.arange(0,df_forcing.shape[0]):
        try:
            print(datetime.now())
            # Latitude and longitude of event being considered
            sel_lat = df_forcing.iloc[i].LAT
            sel_lon = df_forcing.iloc[i].LONG
            sel_lon_360 = df_forcing.iloc[i].LONG % 360
            # Central time of event being considered
            sel_date = df_forcing.iloc[i].DATETIME
            # Event ID
            sel_ID = df_forcing.iloc[i].ID

            # Load full forcing time series at this location
            bfr_all = open_forcing(forcing_name='df_forcing_' + str(sel_lat) + '_' + str(sel_lon_360) + '_' + years[0] + '_' + r_tag + '.csv', forcing_dir=forcing_dir)
        #     print(datetime.now())
            for y in years[1:]:
                bfr_temp = open_forcing(forcing_name='df_forcing_' + str(sel_lat) + '_' + str(sel_lon_360) + '_' + y + '_' + r_tag + '.csv', forcing_dir=forcing_dir)
                bfr_all = pd.concat([bfr_all,bfr_temp])
        #     print(datetime.now())
            # Load full HYCOM time series at this location
            files = [j for j in sorted(glob.glob(HYCOM_dir + 'HYCOM_53X_' + str(sel_lat) + '_' + str(sel_lon) + '*.pkl'))]
            # Open and concatenate all files into a single xarray.Dataset
            data_load = pkl.load(open(files[0], "rb"))
            data_load = xr.concat([pkl.load(open(f, "rb"))for f in files[1:]], "index")
            # Reset index
            data_load['index'] = np.arange(0,data_load.index.shape[0])
            # Define timestamps to keep only HYCOM times of interest (+- n days from sel_date)
            start_date_plot = pd.to_datetime(sel_date) - timedelta(days = days_before_sel_date)
            end_date_plot = pd.to_datetime(sel_date) + timedelta(days = days_after_sel_date) # originally days was 7
            # Select new_dict in time period of event (based on start/end dat plot around event)
            index_masked = data_load.index[((data_load.time >= start_date_plot) & (data_load.time <= end_date_plot))]
            new_dict_period = data_load.sel(index=slice(index_masked[0], index_masked[-1]))
            
            # If all NaNs (i.e., data point on the ground), skip to next
            if np.nansum(new_dict_period.salinity.values) == 0:
                continue
            
            # Define a timestamp for the "before" conditions (-2 days from event center)
            timestep_before = pd.to_datetime(sel_date) - timedelta(days = days_from_event_center)
        #     new_dict_period = new_dict[(new_dict_period.time >= start_date_plot) & (new_dict_period.time <= end_date_plot)]
            # Save variables in the period of interest (+- 7 days from sel_date)
            time_plot = new_dict_period.time #[(new_dict.time >= start_date_plot) & (new_dict.time <= end_date_plot)]
            # Time variable to show time difference from sel_date
            delta_time = pd.to_datetime(time_plot) - pd.to_datetime(sel_date)
            
            # Approximate delta_time to always have a zero
            delta_time_approx = delta_time - max(delta_time[delta_time<=timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)])

        #     temperature_abs_period = new_dict_period.temperature#[(new_dict.time >= start_date_plot) & (new_dict.time <= end_date_plot)]
        #     salinity_abs_period = new_dict_period.salinity#[(new_dict.time >= start_date_plot) & (new_dict.time <= end_date_plot)]

            # Calculate pressure from depth using gsw.p_from_z (TEOS-10 Python Package)
            pressure = gsw.p_from_z(-new_dict_period.depth.values, float(new_dict_period.latitude.values[0]))
            # Calculate absolute salinity from practical salinity    
            new_dict_period["absolute_salinity"] = xr.DataArray(gsw.conversions.SA_from_SP(new_dict_period.salinity.values, pressure, \
                                                                float(new_dict_period.longitude.values[0]), \
                                                                float(new_dict_period.latitude.values[0])), \
                                                                coords=new_dict_period.coords, \
                                                                dims=new_dict_period.dims)
            # Calculate potential temperature from in-situ temperature
            potential_temperature = gsw.conversions.pt0_from_t(new_dict_period["absolute_salinity"].values, new_dict_period.temperature.values, pressure)
            # Calculate conservative temperature from potential temperature
            new_dict_period["conservative_temperature"] = xr.DataArray(gsw.conversions.CT_from_pt(new_dict_period["absolute_salinity"].values, \
                                                                       potential_temperature), \
                                                                       coords=new_dict_period.coords,\
                                                                       dims=new_dict_period.dims)
            # Calculate density using gsw.density.rho (TEOS-10 Python Package)
            # This function returns density in kg/m3
            new_dict_period["potential_density"] = xr.DataArray(gsw.density.sigma0(new_dict_period["absolute_salinity"].values, new_dict_period["conservative_temperature"].values)+1000, 
                                          coords=new_dict_period["absolute_salinity"].coords,
                                          dims=new_dict_period["absolute_salinity"].dims)

            # MIXED LAYER DEPTH
            tempMLD=[]
            salinityMLD = []
            densityMLD = []
            
            for i_t in np.arange(new_dict_period["conservative_temperature"].shape[0]):
                # Remove NaNs to let the function work
                conservative_temperature_nonans = new_dict_period["conservative_temperature"].values[i_t][~np.isnan(new_dict_period["conservative_temperature"].values[i_t])]
                absolute_salinity_nonans = new_dict_period["absolute_salinity"].values[i_t][~np.isnan(new_dict_period["absolute_salinity"].values[i_t])]
                potential_density_nonans = new_dict_period["potential_density"].values[i_t][~np.isnan(new_dict_period["potential_density"].values[i_t])]
                # Apply function
                h = HolteAndTalley(pressures=pressure[:conservative_temperature_nonans.shape[0]], \
                               temperatures=conservative_temperature_nonans, \
                               salinities=absolute_salinity_nonans, \
                               densities=potential_density_nonans)            
                
                ##The temperature algorithms mixed layer depth
                tempMLD.append(h.tempMLD)
                ##The salinity algorithms mixed layer depth
                salinityMLD.append(h.salinityMLD)
                ##The density algorithms mixed layer depth
                densityMLD.append(h.densityMLD)

            new_dict_period["tempMLD"] = xr.DataArray(np.asarray(tempMLD), coords=[new_dict_period.index.values], dims = ["index"])
            new_dict_period["salinityMLD"] = xr.DataArray(np.asarray(salinityMLD), coords=[new_dict_period.index.values], dims = ["index"])
            new_dict_period["densityMLD"] = xr.DataArray(np.asarray(densityMLD), coords=[new_dict_period.index.values], dims = ["index"])

            # Remove seasonal cycle (from each month, remove the long-term mean for that specific month)
            # method from https://earth-env-data-science.github.io/lectures/xarray/xarray-part2.html
            new_dict_anom_period = new_dict_period.groupby(new_dict_period.time.dt.month) - new_dict_period.groupby(new_dict_period.time.dt.month).mean(dim='index')

            # Select precipitation +- 2 days from event center
            precipitation_4days = bfr_all.PRECIPITATION[(bfr_all.DATETIME > pd.to_datetime(sel_date) \
                                                         - timedelta(days = days_from_event_center)) & \
                                                        (bfr_all.DATETIME < pd.to_datetime(sel_date) + \
                                                         timedelta(days = days_from_event_center))]
            time_4days = bfr_all.DATETIME[(bfr_all.DATETIME > pd.to_datetime(sel_date) - \
                                           timedelta(days = days_from_event_center)) & \
                                          (bfr_all.DATETIME < pd.to_datetime(sel_date) + \
                                           timedelta(days = days_from_event_center))]
            # Compute the integral using the composite trapezoidal rule
            integral_precip = trapz(precipitation_4days, dx = 1) # dx = 1 hour

            # Calculate differences against -2 days
            # Start by selecting values of variables at day -2
            time_minus2 = new_dict_period.time[(new_dict_period.time >= timestep_before) & (new_dict_period.time <= end_date_plot)]

            salinity_abs_dayminus2 = new_dict_period["absolute_salinity"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            temperature_abs_dayminus2 = new_dict_period["conservative_temperature"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            density_abs_dayminus2 = new_dict_period["potential_density"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            
            salinityMLD_minus2 = new_dict_period["salinityMLD"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            tempMLD_minus2 = new_dict_period["tempMLD"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            densityMLD_minus2 = new_dict_period["densityMLD"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            
            salinity_anom_dayminus2 = new_dict_anom_period["absolute_salinity"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            temperature_anom_dayminus2 = new_dict_anom_period["conservative_temperature"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            density_anom_dayminus2 = new_dict_anom_period["potential_density"][time_plot == nearest(pd.to_datetime(time_plot),pd.to_datetime(timestep_before))]
            
            # Calculate change in properties with respect to day -2
            # Initialize variables to populate in loop
            salinity_abs_period_diff = np.zeros(new_dict_period["absolute_salinity"].shape)
            temperature_abs_period_diff = np.zeros(new_dict_period["conservative_temperature"].shape)
            density_abs_period_diff = np.zeros(new_dict_period["potential_density"].shape)
            salinityMLD_abs_period_diff = np.zeros(new_dict_period["salinityMLD"].shape)
            tempMLD_abs_period_diff = np.zeros(new_dict_period["tempMLD"].shape)
            densityMLD_abs_period_diff = np.zeros(new_dict_period["densityMLD"].shape)            
            salinity_anom_period_diff = np.zeros(new_dict_anom_period["absolute_salinity"].shape)
            temperature_anom_period_diff = np.zeros(new_dict_anom_period["conservative_temperature"].shape)
            density_anom_period_diff = np.zeros(new_dict_anom_period["potential_density"].shape)
            
            for n in np.arange(0,len(time_plot)):
                salinity_abs_period_diff[n,:] = new_dict_period["absolute_salinity"][n,:] - salinity_abs_dayminus2[0]
                temperature_abs_period_diff[n,:] = new_dict_period["conservative_temperature"][n,:] - temperature_abs_dayminus2[0]
                density_abs_period_diff[n,:] = new_dict_period["potential_density"][n,:] - density_abs_dayminus2[0]

                salinityMLD_abs_period_diff[n] = new_dict_period["salinityMLD"][n] - salinityMLD_minus2[0]
                tempMLD_abs_period_diff[n] = new_dict_period["tempMLD"][n] - tempMLD_minus2[0]
                densityMLD_abs_period_diff[n] = new_dict_period["densityMLD"][n] - densityMLD_minus2[0]

                salinity_anom_period_diff[n,:] = new_dict_anom_period["absolute_salinity"][n,:] - salinity_anom_dayminus2[0]
                temperature_anom_period_diff[n,:] = new_dict_anom_period["conservative_temperature"][n,:] - temperature_anom_dayminus2[0]
                density_anom_period_diff[n,:] = new_dict_anom_period["potential_density"][n,:] - density_anom_dayminus2[0]                
                
            #_______________
            # Save HYCOM data
            d_xr = xr.Dataset(
                data_vars=dict(
                    # Absolute values
                    temperature_abs=(["time","depth"], new_dict_period["conservative_temperature"].values , {'label': 'HYCOM T at TC location'}),
                    salinity_abs=(["time","depth"], new_dict_period["absolute_salinity"].values, {'label': 'HYCOM S at TC location'}),
                    density_abs=(["time","depth"], new_dict_period["potential_density"].values , {'label': 'HYCOM density at TC location'}),
                    
                    # Anomalies
                    temperature_anom=(["time","depth"], new_dict_anom_period["conservative_temperature"].values ,{'label': 'HYCOM T anomalies at TC location'}),
                    salinity_anom=(["time","depth"], new_dict_anom_period["absolute_salinity"].values,{'label': 'HYCOM S anomalies at TC location'}),
                    density_anom=(["time","depth"], new_dict_anom_period["potential_density"].values ,{'label': 'HYCOM density anomalies at TC location'}),
                    
                    # Variations from day -2, absolute values
                    salinity_abs_minus2=(["time","depth"], salinity_abs_period_diff, {'label': 'HYCOM S at TC location, difference from day -2'}),
                    temperature_abs_minus2=(["time","depth"], temperature_abs_period_diff , {'label': 'HYCOM T at TC location, difference from day -2'}),
                    density_abs_minus2=(["time","depth"], density_abs_period_diff , {'label': 'HYCOM density at TC location, difference from day -2'}),

                    # Variations from day -2, anomalies
                    salinity_anom_minus2=(["time","depth"], salinity_anom_period_diff, {'label': 'HYCOM S anomalies at TC location, difference from day -2'}),
                    temperature_anom_minus2=(["time","depth"], temperature_anom_period_diff , {'label': 'HYCOM T anomalies at TC location, difference from day -2'}),
                    density_anom_minus2=(["time","depth"], density_anom_period_diff , {'label': 'HYCOM density anomalies at TC location, difference from day -2'}),
                    
                    # MLD - absolute values
                    salinityMLD_abs = (["time"], new_dict_period["salinityMLD"].values, {'label': 'HYCOM salinityMLD at TC location, difference from day -2'}),
                    tempMLD_abs = (["time"], new_dict_period["tempMLD"].values ,{'label': 'HYCOM temperatureMLD at TC location, difference from day -2'}),
                    densityMLD_abs = (["time"], new_dict_period["densityMLD"].values ,{'label': 'HYCOM densityMLD at TC location, difference from day -2'}),

                    # Variations from day -2 for MLD
                    salinityMLD_minus2 = (["time"], salinityMLD_abs_period_diff, {'label': 'HYCOM salinityMLD at TC location, difference from day -2'}),
                    tempMLD_minus2 = (["time"], tempMLD_abs_period_diff, {'label': 'HYCOM temperatureMLD at TC location, difference from day -2'}),
                    densityMLD_minus2 = (["time"], densityMLD_abs_period_diff, {'label': 'HYCOM densityMLD at TC location, difference from day -2'}),

                    delta_time=(["time"], delta_time ,{'label': 'Delta Time from Center Time of the Event)'}),
                    delta_time_approx=(["time"], delta_time_approx ,{'label': 'Delta Time approximated from Center Time of the Event)'})
                ),
                    coords=dict(
                        time=time_plot.values,
                        depth=new_dict_period.depth.values,
                        longitude = sel_lon,
                        latitude = sel_lat,
                        integral_precip = integral_precip,
                        sel_date = sel_date
                    ),
                    attrs=dict(description="HYCOM T/S co-located with TCs"),
                    )

            with open(HYCOM_save_dir+r_tag+'_event_' + str(sel_ID) + '_with_MLD' + '.pkl', 'wb') as handle:
                pickle.dump(d_xr, handle, protocol=pickle.HIGHEST_PROTOCOL)
            print(datetime.now())
            
            if flag_plots:
                #_______________
                # PLOTS
                if float(sum(sum(np.isnan(salinity_abs_period)))) == 0:

                    # ______________
                    # Make plot # 1 - anomalies
                    f, (a0, a1, a2, a3, a4) = plt.subplots(5, 1, figsize = (12,12), gridspec_kw={'height_ratios': [1, 1, 2, 2, 2]})

                    # FORCING WIND SPEED
                    a0.plot(bfr_all.DATETIME,bfr_all.WIND)
                    a0.axhline(y=wind_threshold, color='r', linestyle='--', label = 'Hurricane limit')
                    a0.set_ylabel('Wind speed [m/s]', fontsize = 12)
                    a0.set_xlim([start_date_plot, end_date_plot]) 
                    a0.legend(fontsize = 12)
                    a0.set_title(f'Forcing wind speed (m/s)', fontsize = 14)

                    # FORCING PRECIPITATION
                    a1.plot(bfr_all.DATETIME,bfr_all.PRECIPITATION)
                    a1.set_ylabel('Precipitation', fontsize = 12)
                    a1.set_xlim([start_date_plot, end_date_plot]) 
                    a1.set_ylim([0, 0.03]) 
                    a1.set_title(f'Forcing precipitation (unit?)', fontsize = 14)

                    # HYCOM TEMPERATURE anomalies
                    norm = mcolors.TwoSlopeNorm(vmin=temperature_anom_period.values.transpose().min(), vmax = temperature_anom_period.values.transpose().max(), vcenter=0)
                    im2 = a2.contourf(time_plot.values,new_dict.depth.values,temperature_anom_period.values.transpose(), levels = 20, cmap="RdBu_r", norm=norm)
                    a2.set_ylabel('Depth (m)', fontsize = 12)
                    a2.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a2.set_xlim([start_date_plot, end_date_plot]) 
                    a2.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a2.invert_yaxis()
                    # Colorbar for temperature
                    divider = make_axes_locatable(a2)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im2, cax=cax, orientation='vertical')
                    a2.set_title(f'HYCOM TEMPERATURE anomaly (deg C)', fontsize = 14)

                    # HYCOM SALINITY anomalies
                    norm = mcolors.TwoSlopeNorm(vmin=salinity_anom_period.values.transpose().min(), vmax = salinity_anom_period.values.transpose().max(), vcenter=0)
                    im3 = a3.contourf(time_plot.values,new_dict.depth.values,salinity_anom_period.values.transpose(), levels = 20, cmap="RdBu_r", norm=norm)
                    a3.set_xlabel('Time (UTC)', fontsize = 12)
                    a3.set_ylabel('Depth (m)', fontsize = 12)
                    a3.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a3.set_xlim([start_date_plot, end_date_plot]) 
                    a3.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a3.invert_yaxis()
                    # Colorbar for salinity
                    divider = make_axes_locatable(a3)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im3, cax=cax, orientation='vertical')
                    a3.set_title(f'HYCOM SALINITY anomaly (psu)', fontsize = 14)

                    # HYCOM DENSITY anomalies
                    norm = mcolors.TwoSlopeNorm(vmin=density_anom_period.values.transpose().min(), vmax = density_anom_period.values.transpose().max(), vcenter=0)
                    im4 = a4.contourf(time_plot.values,new_dict.depth.values,density_anom_period.values.transpose(), levels = 20, cmap="RdBu_r", norm=norm)
                    a4.set_xlabel('Time (UTC)', fontsize = 12)
                    a4.set_ylabel('Depth (m)', fontsize = 12)
                    a4.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a4.set_xlim([start_date_plot, end_date_plot]) 
                    a4.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a4.invert_yaxis()
                    # Colorbar for density
                    divider = make_axes_locatable(a4)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im4, cax=cax, orientation='vertical')
                    a4.set_title(f'HYCOM DENSITY anomaly (kg/m3)', fontsize = 14)


                    a0.xaxis.get_label().set_fontsize(12)
                    a1.xaxis.get_label().set_fontsize(12)
                    a2.xaxis.get_label().set_fontsize(12)
                    a3.xaxis.get_label().set_fontsize(12)
                    a4.xaxis.get_label().set_fontsize(12)
                    a0.yaxis.get_label().set_fontsize(12)
                    a1.yaxis.get_label().set_fontsize(12)
                    a2.yaxis.get_label().set_fontsize(12)
                    a3.yaxis.get_label().set_fontsize(12)
                    a4.yaxis.get_label().set_fontsize(12)
                    a0.tick_params(axis='y', labelsize=12)
                    a1.tick_params(axis='y', labelsize=12)
                    a2.tick_params(axis='y', labelsize=12)
                    a3.tick_params(axis='y', labelsize=12)
                    a4.tick_params(axis='y', labelsize=12)

                    f.tight_layout()
                    f.savefig('./PLOTS/' + folder_tag + '_event_' + str(sel_ID) + '.pdf', dpi = 300)
                    f.savefig('./PLOTS/' + folder_tag + '_event_' + str(sel_ID) + '.png', dpi = 300)
                    # ______________

                    # ______________
                    # Make plot # 2 - absolute values
                    f, (a0, a1, a2) = plt.subplots(3, 1, figsize = (12,12), gridspec_kw={'height_ratios': [2, 2, 2]})

                    # Temperature
                    im0 = a0.contourf(time_plot,new_dict.depth,temperature_abs_period.transpose(), levels = 20)
                    # a0.contour(new_dict.time.values,new_dict.depth.values,new_dict.temperature.values.transpose(), levels = 20, linewidths=.5, colors='black')
                    a0.set_xlabel('Time (UTC)', fontsize = 12)
                    a0.set_ylabel('Depth (m)', fontsize = 12)
                    a0.tick_params(axis='y', labelsize=12)
                    a0.tick_params(axis='x', labelsize=12)
                    a0.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a0.set_xlim([start_date_plot, end_date_plot]) 
                    a0.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a0.invert_yaxis()
                    divider = make_axes_locatable(a0)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im0, cax=cax, orientation='vertical')
                    a0.set_title(f'HYCOM absolute TEMPERATURE (deg C)', fontsize = 14)

                    # Salinity
                    im1 = a1.contourf(time_plot,new_dict.depth,salinity_abs_period.transpose(), levels = 20)
                    # a1.contour(new_dict.time.values,new_dict.depth.values,new_dict.temperature.values.transpose(), levels = 20, linewidths=.5, colors='black')
                    a1.set_xlabel('Time (UTC)', fontsize = 12)
                    a1.set_ylabel('Depth (m)', fontsize = 12)
                    a1.tick_params(axis='y', labelsize=12)
                    a1.tick_params(axis='x', labelsize=12)
                    a1.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a1.set_xlim([start_date_plot, end_date_plot]) 
                    a1.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a1.invert_yaxis()
                    divider = make_axes_locatable(a1)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im1, cax=cax, orientation='vertical')
                    a1.set_title(f'HYCOM absolute SALINITY (psu)', fontsize = 14)

                    # Density
                    im2 = a2.contourf(time_plot,new_dict.depth,density_abs_period.transpose(), levels = 20)
                    # a0.contour(new_dict.time.values,new_dict.depth.values,new_dict.temperature.values.transpose(), levels = 20, linewidths=.5, colors='black')
                    a2.set_xlabel('Time (UTC)', fontsize = 12)
                    a2.set_ylabel('Depth (m)', fontsize = 12)
                    a2.tick_params(axis='y', labelsize=12)
                    a2.tick_params(axis='x', labelsize=12)
                    a2.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a2.set_xlim([start_date_plot, end_date_plot]) 
                    a2.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a2.invert_yaxis()
                    divider = make_axes_locatable(a2)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im2, cax=cax, orientation='vertical')
                    a2.set_title(f'HYCOM absolute DENSITY (kg/m3)', fontsize = 14)

                    f.tight_layout()
                    f.savefig('./PLOTS/' + folder_tag + '_event_' + str(sel_ID) + '_abs.pdf', dpi = 300)
                    f.savefig('./PLOTS/' + folder_tag + '_event_' + str(sel_ID) + '_abs.png', dpi = 300)
                    # ______________


                    # ______________
                    # Make plot # 3 - absolute values relative to day -2
                    f, (a0, a1, a2) = plt.subplots(3, 1, figsize = (12,12), gridspec_kw={'height_ratios': [2, 2, 2]})

                    # Temperature
                    norm = mcolors.TwoSlopeNorm(vmin=temperature_abs_period_diff.transpose().min(), vmax = temperature_abs_period_diff.transpose().max(), vcenter=0)
                    im0 = a0.contourf(time_plot,new_dict.depth,temperature_abs_period_diff.transpose(), levels = 20, cmap="RdBu_r", norm=norm)
                    a0.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a0.set_xlabel('Time (UTC)', fontsize = 12)
                    a0.set_ylabel('Depth (m)', fontsize = 12)
                    a0.tick_params(axis='y', labelsize=12)
                    a0.tick_params(axis='x', labelsize=12)
                    a0.set_xlim([timestep_before, end_date_plot]) 
                    a0.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a0.invert_yaxis()
                    divider = make_axes_locatable(a0)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im0, cax=cax, orientation='vertical')
                    a0.set_title(f'HYCOM absolute TEMPERATURE change from day -2 (deg C)', fontsize = 14)

                    # Salinity
                    norm = mcolors.TwoSlopeNorm(vmin=salinity_abs_period_diff.transpose().min(), vmax = salinity_abs_period_diff.transpose().max(), vcenter=0)
                    im1 = a1.contourf(time_plot,new_dict.depth,salinity_abs_period_diff.transpose(), levels = 20, cmap="RdBu_r", norm=norm)
                    a1.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a1.set_xlabel('Time (UTC)', fontsize = 12)
                    a1.set_ylabel('Depth (m)', fontsize = 12)
                    a1.tick_params(axis='y', labelsize=12)
                    a1.tick_params(axis='x', labelsize=12)
                    a1.set_xlim([timestep_before, end_date_plot]) 
                    a1.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a1.invert_yaxis()
                    divider = make_axes_locatable(a1)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im1, cax=cax, orientation='vertical')
                    a1.set_title(f'HYCOM absolute SALINITY change from day -2 (psu)', fontsize = 14)

                    # Density
                    norm = mcolors.TwoSlopeNorm(vmin=density_abs_period_diff.transpose().min(), vmax = density_abs_period_diff.transpose().max(), vcenter=0)
                    im2 = a2.contourf(time_plot,new_dict.depth,density_abs_period_diff.transpose(), levels = 20, cmap="RdBu_r", norm=norm)
                    a2.axvline(x=pd.to_datetime(sel_date), linestyle = '--', c = 'k', linewidth = 1)
                    a2.set_xlabel('Time (UTC)', fontsize = 12)
                    a2.set_ylabel('Depth (m)', fontsize = 12)
                    a2.tick_params(axis='y', labelsize=12)
                    a2.tick_params(axis='x', labelsize=12)
                    a2.set_xlim([timestep_before, end_date_plot]) 
                    a2.set_ylim([new_dict.depth[0], new_dict.depth[max_depth_index_sel]]) # select the first 200 meters
                    a2.invert_yaxis()
                    divider = make_axes_locatable(a2)
                    cax = divider.append_axes('right', size='2%', pad=0.05)
                    f.colorbar(im2, cax=cax, orientation='vertical')
                    a2.set_title(f'HYCOM absolute DENSITY change from day -2 (deg C)', fontsize = 14)

                    f.tight_layout()
                    f.savefig('./PLOTS/' + folder_tag + '_event_' + str(sel_ID) + '_abs_minus2.pdf', dpi = 300)
                    f.savefig('./PLOTS/' + folder_tag + '_event_' + str(sel_ID) + '_abs_minus2.png', dpi = 300)
                    # ______________
        except:
            pass


2024-07-26 13:44:46.086196
df_forcing_15.229980449149714_128.2499991506338_2011_WPacific_1of3.csv
df_forcing_15.229980449149714_128.2499991506338_2012_WPacific_1of3.csv
df_forcing_15.229980449149714_128.2499991506338_2013_WPacific_1of3.csv
df_forcing_15.229980449149714_128.2499991506338_2014_WPacific_1of3.csv
df_forcing_15.229980449149714_128.2499991506338_2015_WPacific_1of3.csv
2024-07-26 13:44:46.211611
df_forcing_15.025551182985945_134.38636274635792_2011_WPacific_1of3.csv
df_forcing_15.025551182985945_134.38636274635792_2012_WPacific_1of3.csv
df_forcing_15.025551182985945_134.38636274635792_2013_WPacific_1of3.csv
df_forcing_15.025551182985945_134.38636274635792_2014_WPacific_1of3.csv
df_forcing_15.025551182985945_134.38636274635792_2015_WPacific_1of3.csv
2024-07-26 13:44:46.329044
df_forcing_11.345824381493092_125.79545371234416_2011_WPacific_1of3.csv
df_forcing_11.345824381493092_125.79545371234416_2012_WPacific_1of3.csv
df_forcing_11.345824381493092_125.79545371234416_2013_WPacif

df_forcing_18.29641943271801_146.04545357823372_2011_WPacific_1of3.csv
df_forcing_18.29641943271801_146.04545357823372_2012_WPacific_1of3.csv
df_forcing_18.29641943271801_146.04545357823372_2013_WPacific_1of3.csv
df_forcing_18.29641943271801_146.04545357823372_2014_WPacific_1of3.csv
df_forcing_18.29641943271801_146.04545357823372_2015_WPacific_1of3.csv
2024-07-26 13:44:48.956109
df_forcing_16.45655604464892_122.72727191448212_2011_WPacific_1of3.csv
df_forcing_16.45655604464892_122.72727191448212_2012_WPacific_1of3.csv
df_forcing_16.45655604464892_122.72727191448212_2013_WPacific_1of3.csv
df_forcing_16.45655604464892_122.72727191448212_2014_WPacific_1of3.csv
df_forcing_16.45655604464892_122.72727191448212_2015_WPacific_1of3.csv
2024-07-26 13:44:49.080036
df_forcing_12.16354145007256_129.06818096339703_2011_WPacific_1of3.csv
df_forcing_12.16354145007256_129.06818096339703_2012_WPacific_1of3.csv
df_forcing_12.16354145007256_129.06818096339703_2013_WPacific_1of3.csv
df_forcing_12.163541450

df_forcing_16.66098531030771_141.74999906122684_2014_WPacific_1of3.csv
df_forcing_16.66098531030771_141.74999906122684_2015_WPacific_1of3.csv
2024-07-26 13:44:51.707415
df_forcing_10.936965846904853_128.86363551020622_2011_WPacific_1of3.csv
df_forcing_10.936965846904853_128.86363551020622_2012_WPacific_1of3.csv
df_forcing_10.936965846904853_128.86363551020622_2013_WPacific_1of3.csv
df_forcing_10.936965846904853_128.86363551020622_2014_WPacific_1of3.csv
df_forcing_10.936965846904853_128.86363551020622_2015_WPacific_1of3.csv
2024-07-26 13:44:51.830169
df_forcing_20.749570605771247_128.45454460382462_2011_WPacific_1of3.csv
df_forcing_20.749570605771247_128.45454460382462_2012_WPacific_1of3.csv
df_forcing_20.749570605771247_128.45454460382462_2013_WPacific_1of3.csv
df_forcing_20.749570605771247_128.45454460382462_2014_WPacific_1of3.csv
df_forcing_20.749570605771247_128.45454460382462_2015_WPacific_1of3.csv
2024-07-26 13:44:51.955652
df_forcing_22.58943397605009_129.47727186977863_2011_WPac

df_forcing_28.722311803135124_132.34090821444988_2011_WPacific_1of3.csv
df_forcing_28.722311803135124_132.34090821444988_2012_WPacific_1of3.csv
df_forcing_28.722311803135124_132.34090821444988_2013_WPacific_1of3.csv
df_forcing_28.722311803135124_132.34090821444988_2014_WPacific_1of3.csv
df_forcing_28.722311803135124_132.34090821444988_2015_WPacific_1of3.csv
2024-07-26 13:44:54.581189
df_forcing_20.545141341878733_130.49999913573265_2011_WPacific_1of3.csv
df_forcing_20.545141341878733_130.49999913573265_2012_WPacific_1of3.csv
df_forcing_20.545141341878733_130.49999913573265_2013_WPacific_1of3.csv
df_forcing_20.545141341878733_130.49999913573265_2014_WPacific_1of3.csv
df_forcing_20.545141341878733_130.49999913573265_2015_WPacific_1of3.csv
2024-07-26 13:44:54.708894
df_forcing_14.61669265045644_144.8181808590889_2011_WPacific_1of3.csv
df_forcing_14.61669265045644_144.8181808590889_2012_WPacific_1of3.csv
df_forcing_14.61669265045644_144.8181808590889_2013_WPacific_1of3.csv
df_forcing_14.61

df_forcing_27.0868777285458_128.2499991506338_2015_WPacific_1of3.csv
2024-07-26 13:44:57.255838
df_forcing_18.705277962482167_122.1136355549097_2011_WPacific_1of3.csv
df_forcing_18.705277962482167_122.1136355549097_2012_WPacific_1of3.csv
df_forcing_18.705277962482167_122.1136355549097_2013_WPacific_1of3.csv
df_forcing_18.705277962482167_122.1136355549097_2014_WPacific_1of3.csv
df_forcing_18.705277962482167_122.1136355549097_2015_WPacific_1of3.csv
2024-07-26 13:44:57.378779
df_forcing_18.091990167705912_134.59090819954872_2011_WPacific_1of3.csv
df_forcing_18.091990167705912_134.59090819954872_2012_WPacific_1of3.csv
df_forcing_18.091990167705912_134.59090819954872_2013_WPacific_1of3.csv
df_forcing_18.091990167705912_134.59090819954872_2014_WPacific_1of3.csv
df_forcing_18.091990167705912_134.59090819954872_2015_WPacific_1of3.csv
2024-07-26 13:44:57.508506
df_forcing_19.93185354960341_132.7499991208315_2011_WPacific_1of3.csv
df_forcing_19.93185354960341_132.7499991208315_2012_WPacific_1of3

...

df_forcing_22.99829250155351_138.6818172633648_2012_WPacific_1of3.csv
df_forcing_22.99829250155351_138.6818172633648_2013_WPacific_1of3.csv
df_forcing_22.99829250155351_138.6818172633648_2014_WPacific_1of3.csv
df_forcing_22.99829250155351_138.6818172633648_2015_WPacific_1of3.csv
2024-07-26 13:45:08.458562
df_forcing_16.252126778914594_146.86363539099693_2011_WPacific_1of3.csv
df_forcing_16.252126778914594_146.86363539099693_2012_WPacific_1of3.csv
df_forcing_16.252126778914594_146.86363539099693_2013_WPacific_1of3.csv
df_forcing_16.252126778914594_146.86363539099693_2014_WPacific_1of3.csv
df_forcing_16.252126778914594_146.86363539099693_2015_WPacific_1of3.csv
2024-07-26 13:45:08.583158
df_forcing_18.29641943271801_137.24999909102917_2011_WPacific_1of3.csv
df_forcing_18.29641943271801_137.24999909102917_2012_WPacific_1of3.csv
df_forcing_18.29641943271801_137.24999909102917_2013_WPacific_1of3.csv
df_forcing_18.29641943271801_137.24999909102917_2014_WPacific_1of3.csv
df_forcing_18.29641943

df_forcing_19.52299502093509_128.45454460382462_2015_WPacific_1of3.csv
2024-07-26 13:45:11.205246
df_forcing_27.904594767116624_131.72727185487747_2011_WPacific_1of3.csv
df_forcing_27.904594767116624_131.72727185487747_2012_WPacific_1of3.csv
df_forcing_27.904594767116624_131.72727185487747_2013_WPacific_1of3.csv
df_forcing_27.904594767116624_131.72727185487747_2014_WPacific_1of3.csv
df_forcing_27.904594767116624_131.72727185487747_2015_WPacific_1of3.csv
2024-07-26 13:45:11.325368
df_forcing_23.40715102658565_127.2272718846798_2011_WPacific_1of3.csv
df_forcing_23.40715102658565_127.2272718846798_2012_WPacific_1of3.csv
df_forcing_23.40715102658565_127.2272718846798_2013_WPacific_1of3.csv
df_forcing_23.40715102658565_127.2272718846798_2014_WPacific_1of3.csv
df_forcing_23.40715102658565_127.2272718846798_2015_WPacific_1of3.csv
2024-07-26 13:45:11.458271
df_forcing_23.61158028892116_129.68181732296944_2011_WPacific_1of3.csv
df_forcing_23.61158028892116_129.68181732296944_2012_WPacific_1of3.

2024-07-26 13:45:13.958924
df_forcing_25.65587290527447_127.4318173378706_2011_WPacific_1of3.csv
df_forcing_25.65587290527447_127.4318173378706_2012_WPacific_1of3.csv
df_forcing_25.65587290527447_127.4318173378706_2013_WPacific_1of3.csv
df_forcing_25.65587290527447_127.4318173378706_2014_WPacific_1of3.csv
df_forcing_25.65587290527447_127.4318173378706_2015_WPacific_1of3.csv
2024-07-26 13:45:14.087904
df_forcing_21.77171692367752_125.18181735277176_2011_WPacific_1of3.csv
df_forcing_21.77171692367752_125.18181735277176_2012_WPacific_1of3.csv
df_forcing_21.77171692367752_125.18181735277176_2013_WPacific_1of3.csv
df_forcing_21.77171692367752_125.18181735277176_2014_WPacific_1of3.csv
df_forcing_21.77171692367752_125.18181735277176_2015_WPacific_1of3.csv
2024-07-26 13:45:14.206114
df_forcing_26.064731426931072_127.022726431489_2011_WPacific_1of3.csv
df_forcing_26.064731426931072_127.022726431489_2012_WPacific_1of3.csv
df_forcing_26.064731426931072_127.022726431489_2013_WPacific_1of3.csv
df_f

df_forcing_27.904594767116624_128.86363551020622_2011_WPacific_1of3.csv
df_forcing_27.904594767116624_128.86363551020622_2012_WPacific_1of3.csv
df_forcing_27.904594767116624_128.86363551020622_2013_WPacific_1of3.csv
df_forcing_27.904594767116624_128.86363551020622_2014_WPacific_1of3.csv
df_forcing_27.904594767116624_128.86363551020622_2015_WPacific_1of3.csv
2024-07-26 13:45:17.024925
df_forcing_21.77171692367752_132.13636276125908_2011_WPacific_1of3.csv
df_forcing_21.77171692367752_132.13636276125908_2012_WPacific_1of3.csv
df_forcing_21.77171692367752_132.13636276125908_2013_WPacific_1of3.csv
df_forcing_21.77171692367752_132.13636276125908_2014_WPacific_1of3.csv
df_forcing_21.77171692367752_132.13636276125908_2015_WPacific_1of3.csv
2024-07-26 13:45:17.157083
df_forcing_16.66098531030771_137.04545363783836_2011_WPacific_1of3.csv
df_forcing_16.66098531030771_137.04545363783836_2012_WPacific_1of3.csv
df_forcing_16.66098531030771_137.04545363783836_2013_WPacific_1of3.csv
df_forcing_16.6609

df_forcing_27.0868777285458_128.86363551020622_2012_WPacific_1of3.csv
df_forcing_27.0868777285458_128.86363551020622_2013_WPacific_1of3.csv
df_forcing_27.0868777285458_128.86363551020622_2014_WPacific_1of3.csv
df_forcing_27.0868777285458_128.86363551020622_2015_WPacific_1of3.csv
2024-07-26 13:45:19.808809
df_forcing_23.816009551134204_129.88636277616024_2011_WPacific_1of3.csv
df_forcing_23.816009551134204_129.88636277616024_2012_WPacific_1of3.csv
df_forcing_23.816009551134204_129.88636277616024_2013_WPacific_1of3.csv
df_forcing_23.816009551134204_129.88636277616024_2014_WPacific_1of3.csv
df_forcing_23.816009551134204_129.88636277616024_2015_WPacific_1of3.csv
2024-07-26 13:45:19.933622
df_forcing_27.495736248142386_149.11363537609577_2011_WPacific_1of3.csv
df_forcing_27.495736248142386_149.11363537609577_2012_WPacific_1of3.csv
df_forcing_27.495736248142386_149.11363537609577_2013_WPacific_1of3.csv
df_forcing_27.495736248142386_149.11363537609577_2014_WPacific_1of3.csv
df_forcing_27.4957

In [10]:
time_minus2

NameError: name 'time_minus2' is not defined